## 참고자료 모음
https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab

https://yeong-jin-data-blog.tistory.com/entry/BERT%EB%A1%9C-%ED%95%9C%EA%B8%80-%EC%98%81%ED%99%94-%EB%A6%AC%EB%B7%B0-%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D-%ED%95%98%EA%B8%B0

https://inistory.tistory.com/20

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 사전 Install, import

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 81.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 34.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595723 sha256=d89c29264f6829ee769ebb468391ddfb1f53308117e591e0e0aa38fe16854e3e
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 33.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-0xye3wnt
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-0xye3wnt
     |████████████████████████████████| 129 kB 34.1 MB/s 
     |████████████████████████████████| 54.7 MB 38 kB/s 
     |████████████████████████████████| 4.5 MB 33.2 MB/s 
     |████████████████████████████████| 1.2 MB 52.8 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x3a5f2000 @  0x7fdc3b7ee615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 

In [ ]:
import os
import json
import re
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
import pickle

# Data Load

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/0.리뷰 데이터 모음/NLP Data.xlsx')

In [ ]:
data.head()

,Review,Label
0,매번 9시간 넘게 자거나 깊게 잠들지 못하고 수시로 깨는데 이 거 켜 놨을 때 딱...,1
1,처음엔 휴대폰에서 나는 소리가 적응이 되지 않아 머리가 묵직하고 불편한 데 이틀 ...,2
2,그리고 가끔 아침에 일어났을 때 머리가 몽롱 한 데 그 땐 아침밥을 먹거나 커피 ...,1
3,단점 가끔 휴대폰 소프트웨어가 밤에 자기 마음대로 업데이트 되는데 그때 폰이 꺼졌...,0
4,ㅜㅠ 앱에 표시 되는 광고에 대한 평가 란이 있었는데 그런 거 신경 안 쓰고 잠잘...,2


In [ ]:
data.Label.value_counts()

0    3472
1    3249
2     615
Name: Label, dtype: int64

In [ ]:
len(data)

7336

# KoBert

## Train / Test split

In [ ]:
from sklearn.model_selection import train_test_split
                                                         
train, test = train_test_split(data, test_size=0.15, random_state=0)

In [ ]:
train.Label.value_counts()

0    2921
1    2779
2     535
Name: Label, dtype: int64

In [ ]:
test.Label.value_counts()

0    551
1    470
2     80
Name: Label, dtype: int64

In [ ]:
dataset_train = []
for q, label in zip(train['Review'], train['Label'])  :
    data_2 = []
    data_2.append(q)
    data_2.append(str(label))

    dataset_train.append(data_2)

In [ ]:
dataset_test = []
for q, label in zip(test['Review'], test['Label'])  :
    data_3 = []
    data_3.append(q)
    data_3.append(str(label))

    dataset_test.append(data_3)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

6235
1101


## Model

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0") 

#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
device

device(type='cuda', index=0)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train[30]

(array([   2, 4468, 6900, 3647, 5400, 1719, 1772, 5591, 3632, 7096, 3875,
        1996, 3130, 4571, 5585, 6896, 1801, 7082, 4937, 6116, 4249,  517,
        7784, 1763, 6890,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1], dtype=int32),
 array(26, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], dtype=int32),
 1)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {:.4f} train acc {:.4f}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {:.4f}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {:.4f}".format(e+1, test_acc / (batch_id+1)))
    print("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/98 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.0689 train acc 0.3906
epoch 1 train acc 0.9201


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/18 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8690



  0%|          | 0/98 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0049 train acc 1.0000
epoch 5 train acc 0.9968


  0%|          | 0/18 [00:00<?, ?it/s]

epoch 5 test acc 0.8647



## Save model

In [ ]:
'''
with open('model_221001.pickle','wb') as fw:
    pickle.dump(model, fw)
'''

In [ ]:
# model load
'''
with open('model_221001.pickle', 'rb') as f: 
    model = pickle.load(f)
'''

## Model 정확도 계산

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    for i in max_indices:
      predict.append(i)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
model.eval()
labels=[]
predict=[]
test_acc = 0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    for l in label:
      labels.append(l)
    label = label.long().to(device)
    
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
predict[:3]

[tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0')]

In [ ]:
labels[:3]

[tensor(0, dtype=torch.int32),
 tensor(0, dtype=torch.int32),
 tensor(2, dtype=torch.int32)]

In [ ]:
print(len(labels), len(predict))

1101 1101


In [ ]:
cnt0 = 0
cnt1 = 0
cnt2 = 0
test_labels = []
predict_labels=[]

In [ ]:
pdt0 = 0
pdt1 = 0
pdt2 = 0

In [ ]:
for i in range(len(labels)):
  if labels[i] == 0:
    cnt0 +=1
    test_labels.append(0)
  elif labels[i] == 1:
    cnt1+=1
    test_labels.append(1)
  else:
    cnt2+=1
    test_labels.append(2)

In [ ]:
for i in range(len(predict)):
  if predict[i] == 0:
    pdt0+=1
    predict_labels.append(0)
  elif predict[i] == 1:
    pdt1+=1
    predict_labels.append(1)
  else:
    pdt2+=1
    predict_labels.append(2)

In [ ]:
print(cnt0, cnt1, cnt2)
print(pdt0, pdt1, pdt2)

551 470 80
562 475 64


In [ ]:
predict_df = pd.DataFrame({'Test Label' : test_labels,
              'Predict Label': predict_labels})

In [ ]:
sum(predict_df['Test Label']==predict_df['Predict Label'])/len(predict_df)

0.8619436875567665

### 0 (부정) 예측 정확도

In [ ]:
print(f"0 총 개수: {len(predict_df[predict_df['Test Label']==0])}\n0 예측성공: {sum(predict_df[predict_df['Test Label']==0].iloc[:,1]==0)}")
print('0 정확도: ', round(500/551,4))

0 총 개수: 551
0 예측성공: 500
0 정확도:  0.9074


### 1 (긍정) 예측 정확도

In [ ]:
print(f"1 총 개수: {len(predict_df[predict_df['Test Label']==1])}\n1 예측성공: {sum(predict_df[predict_df['Test Label']==1].iloc[:,1]==1)}")
print('1 정확도: ', round(421/470,4))

1 총 개수: 470
1 예측성공: 421
1 정확도:  0.8957


### 2 (일반) 예측 정확도

In [ ]:
print(f"2 총 개수: {len(predict_df[predict_df['Test Label']==2])}\n2 예측성공: {sum(predict_df[predict_df['Test Label']==2].iloc[:,1]==2)}")
print('2 정확도: ', round(28/80,4))

2 총 개수: 80
2 예측성공: 28
2 정확도:  0.35


In [ ]:
print(f"2를 1로 예측: {sum(predict_df[predict_df['Test Label']==2].iloc[:,1]==1)}")
print(f"2를 0로 예측: {sum(predict_df[predict_df['Test Label']==2].iloc[:,1]==0)}")
## 일반 문장을 긍정, 부정으로 골고루 잘못 판단함

2를 1로 예측: 23
2를 0로 예측: 29
